### 1. Preprocess the data such as highly-variable gene filtering

See https://scanpy-tutorials.readthedocs.io/en/latest/pbmc3k.html for preprocessing

<span style="color:yellow">**Attemp 1:**</span> filter multiple AnnData objects from one dataset

In [4]:
# Environment set-up
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as anndata

In [6]:
from pathlib import Path

dataset_directory = Path('/QRISdata/Q6104/Xiaohan/2_AnnData_objs/GSE132509')
for adata_path in dataset_directory.iterdir():
    if "_uni.h5ad" in adata_path.name:
        adata = anndata.read_h5ad(adata_path)
        print(adata)

### 2. Integration with Scanorama

See https://github.com/brianhie/scanorama for more info